In [1]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["figure.dpi"] = 150
rcParams["savefig.dpi"] = 150

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import approx_fprime
from scipy.linalg import solve_triangular, cho_solve

import hemcee

As a demo, we'll sample from a 10-dimensional covariant Gaussian.
First, we need to define the log probability function and its gradient:

In [3]:
# Generate a random covariance matrix
np.random.seed(42)
ndim = 10
L = np.random.randn(ndim, ndim)
L[np.diag_indices_from(L)] = np.exp(L[np.diag_indices_from(L)])
L[np.triu_indices_from(L, 1)] = 0.0
cov = np.dot(L, L.T)

def logprob(params):
    alpha = solve_triangular(L, params, lower=True)
    return -0.5*np.dot(alpha, alpha)

def grad_logprob(params):
    return -cho_solve((L, True), params)

# Check the gradient numerically
p0 = np.random.multivariate_normal(np.zeros(ndim), cov)
print("Maximum gradient error: {0}".format(
    np.max(np.abs(approx_fprime(p0, logprob, 1e-8) - grad_logprob(p0)))))

Maximum gradient error: 1.1157472671552426e-05


Then we set up the sampler using these functions:

In [4]:
# Choose a dense metric that we will tune
metric = hemcee.metric.DenseMetric(np.eye(ndim))

# We will also tune the step size
step = hemcee.step_size.VariableStepSize()

# Set up the sampler
sampler = hemcee.NoUTurnSampler(logprob, grad_logprob, step_size=step, metric=metric)

All Hamiltonian samplers require a tuning phase (often called "warmup" or "burn in").
During this phase, the step size and metric are automatically tuned:

In [5]:
coords = np.random.randn(ndim)
results = sampler.run_warmup(coords, 5000)

initial warm up: step_size: 2.8e-02; mean(accept_stat): 0.482: 100%|██████████| 100/100 [00:00<00:00, 292.35it/s]
warm up 1/8: step_size: 1.0e-02; mean(accept_stat): 0.469: 100%|██████████| 25/25 [00:00<00:00, 304.89it/s]
warm up 2/8: step_size: 9.1e-01; mean(accept_stat): 0.509: 100%|██████████| 25/25 [00:00<00:00, 305.51it/s]
warm up 3/8: step_size: 1.6e+00; mean(accept_stat): 0.490: 100%|██████████| 50/50 [00:00<00:00, 460.63it/s]
warm up 4/8: step_size: 2.4e+00; mean(accept_stat): 0.497: 100%|██████████| 100/100 [00:00<00:00, 660.03it/s]
warm up 5/8: step_size: 6.5e-01; mean(accept_stat): 0.499: 100%|██████████| 200/200 [00:00<00:00, 607.44it/s]
warm up 6/8: step_size: 1.3e+00; mean(accept_stat): 0.499: 100%|██████████| 400/400 [00:00<00:00, 893.68it/s]
warm up 7/8: step_size: 1.3e+00; mean(accept_stat): 0.499: 100%|██████████| 800/800 [00:00<00:00, 870.41it/s]
warm up 8/8: step_size: 1.3e+00; mean(accept_stat): 0.500: 100%|██████████| 3200/3200 [00:04<00:00, 765.98it/s]
final warm

After burning in, we can run the production MCMC chain:

In [6]:
coords_chain, logprob_chain = sampler.run_mcmc(results[0], 5000, initial_log_prob=results[1])

step_size: 8.9e-01; mean(accept_stat): 0.674: 100%|██████████| 5000/5000 [00:05<00:00, 901.41it/s]


Now, let's check the autocorrelation times:

In [13]:
taus = np.array([hemcee.autocorr.integrated_time(coords_chain[:, i])[0] for i in range(ndim)])
print("Mean autocorrelation time: {0}".format(np.mean(taus)))

Mean autocorrelation time: 1.1840161367165014
